# Библиотеки

In [246]:
import numpy as np
from random import shuffle
from PIL.Image import open
from os import listdir
from typing import List
import sys
np.set_printoptions(threshold=sys.maxsize)

# Класс нейронки

In [247]:
class KN:
    lr = 0.6

    def __init__(self, input, clasters): 
        self.weights = np.random.uniform(low=-0.3, high=0.3, size=(clasters, input))
        self.weights = self.weights.round(2)

    def predict(self, vector: np.ndarray):
        res: np.ndarray =  np.power((vector - self.weights), 2).sum(axis=1)
        winner_index = res.argmin()
        return winner_index

    def train(self, vector: np.ndarray):
        winner_index = self.predict(vector)
        delta: np.ndarray = self.lr * (vector - self.weights[winner_index])
        self.weights[winner_index] += delta
        return np.abs(delta)

# Загрузка датасета

In [248]:
def normalize(image: np.ndarray):
    new_image = []
    for rgb in image:
        rgb: np.ndarray
        if (rgb == [255,255,255]).all():
            new_image.append(0)
        else:
            new_image.append(1)

    return np.array(new_image)

In [249]:
dataset: List[tuple] = []
for file in listdir('data'):
    image = np.array(open(f'data/{file}'))
    x_max, y_max, _ = image.shape
    image = image.reshape((x_max*y_max, 3))
    image = normalize(image)
    dataset.append(tuple((file, image)))

# Обучение

In [250]:
kn = KN(2500, 2)
epoch = 200
kn.lr = 0.9
all_deltas = []
delta = np.zeros(shape=2500) 
epoch_count = 0

for i in range(epoch):
    shuffle(dataset)
    for _, image in dataset:
        delta += kn.train(image)

    delta /= len(dataset)
    delta = np.average(delta)
    all_deltas.append(delta)
    if (delta < 0.05): break

    epoch_count += 1
    kn.lr *= 0.9

# Кластеры

In [251]:
for filename, image in dataset:
    clusster = kn.predict(image)
    print(f'Изображение {filename}: Кластер {clusster}')

print(np.round(all_deltas, 5))
print(epoch_count)

Изображение круг (8).png: Кластер 0
Изображение круг (3).png: Кластер 0
Изображение круг (17).png: Кластер 0
Изображение крест (3).png: Кластер 1
Изображение круг (4).png: Кластер 0
Изображение круг (5).png: Кластер 0
Изображение крест (20).png: Кластер 1
Изображение крест (16).png: Кластер 1
Изображение крест (5).png: Кластер 1
Изображение крест (6).png: Кластер 1
Изображение круг (20).png: Кластер 0
Изображение круг (15).png: Кластер 0
Изображение крест (2).png: Кластер 1
Изображение круг (2).png: Кластер 0
Изображение крест (8).png: Кластер 1
Изображение круг (14).png: Кластер 0
Изображение крест (19).png: Кластер 1
Изображение крест (9).png: Кластер 1
Изображение круг (13).png: Кластер 0
Изображение крест (1).png: Кластер 1
Изображение крест (4).png: Кластер 1
Изображение крест (7).png: Кластер 1
Изображение круг (6).png: Кластер 0
Изображение крест (12).png: Кластер 1
Изображение крест (18).png: Кластер 1
Изображение круг (1).png: Кластер 0
Изображение крест (15).png: Кластер 1
Из